## Load Data

In [1]:
import json

with open('user.json', 'r') as f:
    json_user = json.load(f)

with open('Daily.json', 'r') as f:
    json_daily = json.load(f)

In [2]:
import pandas as pd
patients = pd.DataFrame(json_user['data'], dtype='object')
patients.columns

Index(['displayNameTh', 'Username', 'Email', 'Type', 'Age', 'OrgCompany',
       'LineID', 'EmpType', 'Phone', 'Gender', 'Blood', 'MedicalStaff',
       'BloodResults', 'Home', 'Provice'],
      dtype='object')

In [3]:
patients['Type'].unique()

array(['employee', 'student'], dtype=object)

In [4]:
patients['MedicalStaff'].unique()

array(['ไม่ได้เป็นบุคลากรทางการแพทย์', None, 'บุคลากรทางการแพทย์'],
      dtype=object)

In [5]:
patients['IsMedStaff'] = patients['MedicalStaff'].apply(lambda x: x == 'บุคลากรทางการแพทย์')

In [6]:
patients.dtypes

displayNameTh    object
Username         object
Email            object
Type             object
Age              object
OrgCompany       object
LineID           object
EmpType          object
Phone            object
Gender           object
Blood            object
MedicalStaff     object
BloodResults     object
Home             object
Provice          object
IsMedStaff         bool
dtype: object

In [7]:
patients['BloodResults'].unique()

array(['ไม่เคย', None, '', 'ผลเลือดเป็นบวก (+)'], dtype=object)

In [8]:
patients.shape

(1083, 16)

In [9]:
patients['Username'].unique().shape

(1083,)

In [10]:
dailyforms = pd.DataFrame(json_daily['data'])
dailyforms.columns

Index(['formSaveDaily1', 'formSaveDaily2', 'formSaveDaily2Other',
       'formSaveDaily3', 'formSaveDaily4', 'formSaveDaily5', 'Temperature',
       'LineID', 'displayNameTh', 'displayNameEn', 'Username', 'Email', 'Type',
       'birthdate', 'Age', 'Department', 'OrgCompany', 'Phone', 'Gender',
       'Blood', 'MedicalStaff', 'BloodResults', 'Home', 'COLOR',
       'Create_date'],
      dtype='object')

In [11]:
dailyforms.shape

(389, 25)

In [12]:
dailyforms['COLOR'].unique()

array(['สีเขียว', 'สีเหลือง', 'สีแดง', 'สีส้ม', 'สีน้ำเงิน'], dtype=object)

## Convert to Graph Database

In [13]:
from neo4j import GraphDatabase, exceptions

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "1234"), encrypted=False)

In [14]:
with driver.session() as session:
    session.run("MATCH (n) DETACH DELETE n;")
    session.run("""CREATE CONSTRAINT unique_username
                   ON (n:Person) 
                   ASSERT n.username IS UNIQUE""")

ให้โยนข้อมูลใส่ SQL ก่อนแล้วค่อยสร้าง Graph Database โดยส่งเฉพาะข้อมูลที่จำเป็น (เปิดเผยได้ เพราะ กราฟจะเปิด public) พร้อม id เพื่อให้สามารถ link กลับไปหาข้อมูลใน SQL ได้

In [15]:
def create_person(driver, index, param):
    param["id"] = index
    try:
        with driver.session() as session:
            return session.run("""CREATE (a:Person {id:$id,
                                                    name:$Username, 
                                                    type:$Type,
                                                    age:$Age,
                                                    org:$OrgCompany,
                                                    emptype:$EmpType,
                                                    gender:$Gender,
                                                    blood:$Blood,
                                                    ismedstaff:$IsMedStaff,
                                                    bloodres:$BloodResults,
                                                    home:$Home,
                                                    province:$Provice
                                                   })""", param)
    except exceptions.ConstraintError:
        print('Node exists')
        return None

In [16]:
for index, row in patients.iterrows():
    create_person(driver, index, row.to_dict())

In [17]:
from datetime import datetime, timedelta
def convert_str_date(s):
    return datetime.strptime(s, '%Y-%m-%d')

COLOR_MAP = {'สีเขียว':0, 'สีเหลือง':1, 'สีแดง':4, 'สีส้ม':3, 'สีน้ำเงิน':2, 'สีม่วง':5}
COLOR_MAP_TH_ENG = {'สีเขียว':'Green', 'สีเหลือง':'Yellow', 'สีแดง':'Red', 'สีส้ม':'Orange', 'สีน้ำเงิน':'Blue', 'สีม่วง':'Purple'}

In [18]:
def update_color(driver, param, DATE_THR=timedelta(days=14)):
    with driver.session() as session:
        tx = session.begin_transaction()
        rec = tx.run("MATCH (a:Person {name:$Username } ) "
                    "RETURN id(a), a.name, a.color, a.color_date", param).single()
        if rec:
            needUpdate = True
            if rec['a.color'] and rec['a.color_date']:    
                # if new value has higher color level
                if (COLOR_MAP[rec['a.color']] < COLOR_MAP[param['COLOR']]):
                    # check if new value create date is less than threshold
                    if datetime.now() - convert_str_date(param['Create_date']) < DATE_THR:
                        needUpdate = True;
                    else:
                        needUpdate = False
                # to reduce level, must pass date threshold and new date is greater
                elif (COLOR_MAP[rec['a.color']] > COLOR_MAP[param['COLOR']]):
                    if datetime.now() - convert_str_date(rec['a.color_date']) > DATE_THR and \
                            convert_str_date(rec['a.color_date']) < convert_str_date(param['Create_date']):
                        needUpdate = True;
                    else:
                        needUpdate = False
                # if equal level, keep the lowest date
                else:
                    if convert_str_date(rec['a.color_date']) > convert_str_date(param['Create_date']):
                        needUpdate = True;
                    else:
                        needUpdate = False
                
                if needUpdate:
                    print(f"{rec['id(a)']} update {rec['a.color']:10s} --> {param['COLOR']}")
                    print('\t', rec['a.color_date'], ' \t   \t', param['Create_date'])
                
            if needUpdate:
                tx.run("MATCH (a:Person {name:$Username } ) "
                       "SET a.color = $COLOR, a.color_date = $Create_date ", param)             
        else:
            print('No Node with the Name')
        tx.commit()
        return rec

In [19]:
for index, row in dailyforms.iterrows():
    update_color(driver, row.to_dict())

224 update สีเขียว    --> สีเหลือง
	 2020-03-26  	   	 2020-03-26
230 update สีเขียว    --> สีเหลือง
	 2020-03-23  	   	 2020-03-25
580 update สีเขียว    --> สีเหลือง
	 2020-03-23  	   	 2020-03-25
651 update สีเขียว    --> สีเหลือง
	 2020-03-23  	   	 2020-03-24
685 update สีเหลือง   --> สีส้ม
	 2020-03-23  	   	 2020-03-26
685 update สีส้ม      --> สีแดง
	 2020-03-26  	   	 2020-03-26
686 update สีเขียว    --> สีเหลือง
	 2020-03-23  	   	 2020-03-24
695 update สีเขียว    --> สีเหลือง
	 2020-03-24  	   	 2020-03-24
1929 update สีเหลือง   --> สีแดง
	 2020-03-24  	   	 2020-03-26
214 update สีเขียว    --> สีเหลือง
	 2020-03-27  	   	 2020-03-27


## Convert Graph to JSON

In [20]:
def get_node_info(node, gr):
    if 'Location' in gr:
        return {"id": node['id'], 
                "group": gr,
                "label": node['name'], 
               }
    elif 'Person' in gr:
        return {"id": node['id'],
                "group": gr, 
                "label": node['id'],
                "color": node['color'],
                "type" : node['type'],
                "age" : node['age'],
                "org" : node['org'],
                "emptype" : node['emptype'],
                "gender" : node['gender'],
                "blood" : node['blood'],
                "ismedstaff" : node['ismedstaff'],
                "bloodres" : node['bloodres'],
                "province" : node['province']
               }
    else:
        raise Exceptions('Unknown Label')
    
    

def get_edge_info(edge, p_info, m_info):
    if edge.type == 'WENT_TO':
        return {"from": p_info["id"], "to": m_info["id"], "label": edge.type, "start":edge["start"], "end":edge["end"]}
    else:
        return {"from": p_info["id"], "to": m_info["id"], "label": edge.type}

In [21]:
with driver.session() as session:
    result = session.run("""MATCH (p:Person)
                            OPTIONAL MATCH (p:Person)-[a]->(m)
                            RETURN  p, labels(p), a, m, labels(m);""")

print(result.summary().result_available_after)
print(result.summary().result_consumed_after)
    
nodes = []
edges = []
physics = True

for record in result:
    p_info = get_node_info(record['p'], record['labels(p)'][0])
    
    if p_info not in nodes:
        nodes.append(p_info)
         
    if record['m']:
        m_info = get_node_info(record['m'], record['labels(m)'][0])
        if m_info not in nodes:
            nodes.append(m_info)

        edges.append(get_edge_info(record['a'], p_info, m_info))

0
18


In [22]:
import json
# json.dumps({'nodes':nodes, 'edges':edges})
with open('cc.json', 'w') as outfile:
    json.dump({'nodes':nodes, 'edges':edges}, outfile)